In [1]:
from keras.models import Model
from keras.layers import Lambda,Activation,Dense,Conv2D,Input,BatchNormalization,MaxPooling2D,Flatten
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
input_tensor_1 = Input([32,32,3])
input_tensor_2 = Input([4,])
input_target = Input([2,])

In [3]:
def cus_loss1(y_true,y_pred):
    return K.mean(K.abs(y_true - y_pred))

def cus_loss2(y_true,y_pred):
    return K.mean(K.abs(y_true - y_pred))

In [4]:
#网络结构
# 第一组
x = BatchNormalization(axis=1)(input_tensor_1)

x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(filters=32,kernel_size=(3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x=Flatten()(x)
x=Dense(units=16)(x)
out2 = Dense(units=2)(x)


# 第二组
y = Dense(units=32)(input_tensor_2)
out1 = Dense(units=2)(y)

# 第三组
z = Dense(units=8)(input_target)
out3 = Dense(units=2)(z)


#自定义loss
loss1 = Lambda(lambda x:cus_loss1(*x),name='loss1')([out2,out1]) # out2-out1 = loss
loss2 = Lambda(lambda x:cus_loss2(*x),name='loss2')([out3,out2])

In [5]:
#model = Model(inputs=[input_tensor_1,input_tensor_2,input_target],outputs=[out1,out2,out3])

model = Model(inputs=[input_tensor_1,input_tensor_2,input_target],outputs=[out1,out2,out3,loss1,loss2])

In [6]:
#获取loss，创建loss值
loss_layer1 = model.get_layer('loss1').output
loss_layer2 = model.get_layer('loss2').output

# 向模型中添加loss
model.add_loss(loss_layer1)
model.add_loss(loss_layer2)

In [7]:
#loss + None 表示不需要梯度返回，有多少个out就有多少个none
model.compile(optimizer='sgd',loss=[None,None,None,None,None])

In [8]:
def data_gen(number):
    for i in range(number):
        yield [np.random.normal(1,1,size=(1,32,32,3)),
               np.random.normal(1,1,size=(1,4)),
              np.random.normal(1,1,size=(1,2))],[]
                               
                               
dataset = data_gen(1000)
    

In [9]:
#dataset.__next__()

In [10]:
#train = model.fit_generator(dat)
train = model.fit_generator(dataset,epochs=10,steps_per_epoch=20)

Epoch 1/10
20/20 [==============================] - 1s 62ms/step - loss: 1.9595
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.6639
Epoch 3/10
20/20 [==============================] - 0s 7ms/step - loss: 0.3576
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.2062
Epoch 5/10
20/20 [==============================] - 0s 7ms/step - loss: 0.1539
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.1489
Epoch 7/10
20/20 [==============================] - 0s 8ms/step - loss: 0.1301
Epoch 8/10
20/20 [==============================] - 0s 8ms/step - loss: 0.1465
Epoch 9/10
20/20 [==============================] - 0s 9ms/step - loss: 0.1327
Epoch 10/10
20/20 [==============================] - 0s 8ms/step - loss: 0.1537


In [11]:
from keras.utils.vis_utils import plot_model

plot_model(model=model,to_file='model_plain_loss.png',show_shapes= True)

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.